In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

In [15]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import dask
dask.__version__

'2022.03.0'

In [3]:
xgb.__version__

'1.6.1'

In [4]:
!nvidia-smi

Sat May 21 17:36:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   32C    P8    28W / 280W |      5MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0 Off |                  N/A |
| 40%   

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
train_x = dd.read_csv('../input/xgtrain.csv')
test_x = dd.read_csv('../input/xgval.csv')

In [7]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [8]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f",
                                                 "GPU-50aeb092-88f5-4e0b-7f73-32741666f319"])
client = Client(cluster)


2022-05-21 17:36:40,022 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-21 17:36:40,054 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [9]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [10]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [11]:
num_round = 1000

In [16]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3, 1.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 20, 29),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'eval_metric': trial.suggest_categorical('eval_metric',['merror']),
        'num_class': 10,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()
    
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy_score(test_y, predictions)
    
    return acc

In [17]:
study = optuna.create_study(direction='maximize')

[I 2022-05-22 08:02:47,222] A new study created in memory with name: no-name-3a7405f9-f97f-492d-b390-b6ee988135b6


In [18]:
%%time
study.optimize(objective, n_trials=2)

[08:02:48] task [xgboost.dask]:tcp://127.0.0.1:40265 got new rank 0
[08:02:48] task [xgboost.dask]:tcp://127.0.0.1:40919 got new rank 1
[I 2022-05-22 08:09:38,257] Trial 0 finished with value: 0.9705952380952381 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.14046495813662443, 'alpha': 3.7824174886934343, 'colsample_bytree': 0.77950904227617, 'subsample': 0.7881202074809254, 'learning_rate': 0.031863014544941025, 'max_depth': 21, 'min_child_weight': 7, 'eval_metric': 'merror'}. Best is trial 0 with value: 0.9705952380952381.
[08:09:38] task [xgboost.dask]:tcp://127.0.0.1:40265 got new rank 0
[08:09:38] task [xgboost.dask]:tcp://127.0.0.1:40919 got new rank 1
[I 2022-05-22 08:17:32,783] Trial 1 finished with value: 0.9670238095238095 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.8058429585475075, 'alpha': 0.001740470622659711, 'colsample_bytree': 0.9279393129149021, 'subsample': 0.7726710230709537, 'learnin

CPU times: user 24.2 s, sys: 12.7 s, total: 37 s
Wall time: 14min 44s


In [24]:
%%time
study.optimize(objective, n_trials=50)

[14:26:51] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 0
[14:26:51] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 1
[I 2022-05-16 14:29:45,448] Trial 2 finished with value: 0.9289285714285714 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.005919041577580791, 'alpha': 0.0820343866494112, 'colsample_bytree': 0.6417881674023326, 'subsample': 0.5730178342410559, 'learning_rate': 0.06801008292366051, 'max_depth': 20, 'min_child_weight': 212, 'eval_metric': 'merror'}. Best is trial 2 with value: 0.9289285714285714.
[14:29:45] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 0
[14:29:45] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 1
[I 2022-05-16 14:31:15,819] Trial 3 finished with value: 0.9291666666666667 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 1.5431089523933939, 'alpha': 0.003280718225373605, 'colsample_bytree': 0.43859313480421047, 'subsample': 0.6668485470994432, 'lear

CPU times: user 24min 49s, sys: 10min 57s, total: 35min 46s
Wall time: 16h 24min 27s


In [25]:
%%time
study.optimize(objective, n_trials=10)

[09:19:51] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 0
[09:19:51] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 1
[I 2022-05-17 09:30:30,801] Trial 52 finished with value: 0.9723809523809523 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.0014918933930989308, 'alpha': 2.50723010104015, 'colsample_bytree': 0.7269281331092075, 'subsample': 0.9178577889195211, 'learning_rate': 0.07766833785680123, 'max_depth': 22, 'min_child_weight': 9, 'eval_metric': 'merror'}. Best is trial 48 with value: 0.9726190476190476.
[09:30:30] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 0
[09:30:30] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 1
[I 2022-05-17 09:40:54,663] Trial 53 finished with value: 0.9685714285714285 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.0015644568094683002, 'alpha': 2.826455314081641, 'colsample_bytree': 0.7209337171464871, 'subsample': 0.9745396297675913, 'learn

CPU times: user 5min 1s, sys: 2min 2s, total: 7min 4s
Wall time: 3h 9min 47s


In [26]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softmax']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3, 1.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 20, 50),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'eval_metric': trial.suggest_categorical('eval_metric',['merror']),
        'num_class': 10,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    acc = accuracy_score(test_y, predictions)
    
    return acc

In [ ]:
%%time
study.optimize(objective, n_trials=10)

/usr/local/lib/python3.8/dist-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
[12:32:53] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 0
[12:32:53] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 1
[I 2022-05-17 22:40:33,917] Trial 62 finished with value: 0.9728571428571429 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.00849419392820182, 'alpha': 1.758134942739422, 'colsample_bytree': 0.7651183704243859, 'subsample': 0.8252661565079328, 'learning_rate': 0.05374623776610904, 'max_depth': 28, 'min_child_weight': 7, 'eval_metric': 'merror'}. Best is trial 61 with value: 0.9732142857142857.
/usr/local/lib/python3.8/dist-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
[22:40:33] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 0
[22:40:3

In [28]:
1

1

In [29]:
2

2

In [30]:
3

3

In [31]:
study.best_params

{'objective': 'multi:softmax',
 'tree_method': 'gpu_hist',
 'lambda': 0.0022882170229771605,
 'alpha': 0.23689051402096456,
 'colsample_bytree': 0.8304416579518725,
 'subsample': 0.7626959348447965,
 'learning_rate': 0.052451504665423504,
 'max_depth': 28,
 'min_child_weight': 6,
 'eval_metric': 'merror'}

In [32]:
study.best_value

0.9735714285714285

In [33]:
study.best_trial

FrozenTrial(number=66, values=[0.9735714285714285], datetime_start=datetime.datetime(2022, 5, 19, 5, 5, 15, 164730), datetime_complete=datetime.datetime(2022, 5, 19, 15, 13, 14, 780381), params={'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.0022882170229771605, 'alpha': 0.23689051402096456, 'colsample_bytree': 0.8304416579518725, 'subsample': 0.7626959348447965, 'learning_rate': 0.052451504665423504, 'max_depth': 28, 'min_child_weight': 6, 'eval_metric': 'merror'}, distributions={'objective': CategoricalDistribution(choices=('multi:softmax',)), 'tree_method': CategoricalDistribution(choices=('gpu_hist',)), 'lambda': LogUniformDistribution(high=1.0, low=0.001), 'alpha': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': UniformDistribution(high=1.0, low=0.7), 'subsample': UniformDistribution(high=1.0, low=0.7), 'learning_rate': LogUniformDistribution(high=0.1, low=0.001), 'max_depth': IntUniformDistribution(high=50, low=20, step=1), 'min_child_weigh

In [34]:
study.best_trials

[FrozenTrial(number=66, values=[0.9735714285714285], datetime_start=datetime.datetime(2022, 5, 19, 5, 5, 15, 164730), datetime_complete=datetime.datetime(2022, 5, 19, 15, 13, 14, 780381), params={'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.0022882170229771605, 'alpha': 0.23689051402096456, 'colsample_bytree': 0.8304416579518725, 'subsample': 0.7626959348447965, 'learning_rate': 0.052451504665423504, 'max_depth': 28, 'min_child_weight': 6, 'eval_metric': 'merror'}, distributions={'objective': CategoricalDistribution(choices=('multi:softmax',)), 'tree_method': CategoricalDistribution(choices=('gpu_hist',)), 'lambda': LogUniformDistribution(high=1.0, low=0.001), 'alpha': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': UniformDistribution(high=1.0, low=0.7), 'subsample': UniformDistribution(high=1.0, low=0.7), 'learning_rate': LogUniformDistribution(high=0.1, low=0.001), 'max_depth': IntUniformDistribution(high=50, low=20, step=1), 'min_child_weig